# CONVERT S2 (10m) and MODIS (500m) to DF and join

In [2]:

import rasterio
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point
def raster_to_dataframe_with_metadata(raster_path, drop_nodata=True):
    """
    Converts a multi-band raster to a pandas DataFrame with pixel coordinates,
    band values, and acquisition date extracted from the filename (YYYY-MM-DD).

    Args:
        raster_path (str): Path to the raster file.
        drop_nodata (bool): If True, drops pixels with any no-data values.

    Returns:
        pd.DataFrame: DataFrame with columns for x, y, bands, and date.
    """
    with rasterio.open(raster_path) as src:
        bands = src.read()  # (bands, height, width)
        transform = src.transform
        nodata = src.nodata
        count = src.count

        # Try to get band names from metadata
        band_names = list(src.descriptions)
        if not all(band_names):
            band_names = [f'band_{i+1}' for i in range(count)]

        # Get pixel coordinates
        rows, cols = np.meshgrid(np.arange(src.height), np.arange(src.width), indexing='ij')
        xs, ys = rasterio.transform.xy(transform, rows, cols)
        xs = np.array(xs).flatten()
        ys = np.array(ys).flatten()

        # Flatten band data
        band_data = bands.reshape(count, -1).T  # (n_pixels, n_bands)

        # Extract date from filename (format: S2_YYYY-MM-DD)
        filename = os.path.basename(raster_path)
        match = re.search(r"\d{4}-\d{2}-\d{2}", filename)
        date_obj = datetime.strptime(match.group(), "%Y-%m-%d") if match else None

        # Build DataFrame
        df = pd.DataFrame(band_data, columns=band_names)
        df["x"] = xs
        df["y"] = ys
        df["date"] = date_obj

        # Drop NoData pixels
        if drop_nodata and nodata is not None:
            df = df[~np.any(band_data == nodata, axis=1)]
        elif drop_nodata:
            df = df.dropna()

        return df


In [3]:
df_S2 = raster_to_dataframe_with_metadata(r"C:\test\S2_2021-01-13.tif")
df_MODIS = raster_to_dataframe_with_metadata(r"C:\test\MODIS_2021-01-13.tif")
df_MODIS


,MODIS_Albedo_WSA_shortwave,x,y,date
548,479.0,-9243859.354,4.942354e+06,2021-01-13
658,472.0,-9244359.354,4.941854e+06,2021-01-13
767,465.0,-9245359.354,4.941354e+06,2021-01-13
876,510.0,-9246359.354,4.940854e+06,2021-01-13
877,564.0,-9245859.354,4.940854e+06,2021-01-13
...,...,...,...,...
7446,504.0,-9291359.354,4.910854e+06,2021-01-13
7447,494.0,-9290859.354,4.910854e+06,2021-01-13
7448,483.0,-9290359.354,4.910854e+06,2021-01-13
7450,634.0,-9289359.354,4.910854e+06,2021-01-13


In [ ]:

df_S2['geometry'] = df_S2.apply(lambda row: Point(row['x'], row['y']), axis=1)

# Step 2: Convert to GeoDataFrame with Sinusoidal projection
df_S2 = gpd.GeoDataFrame(df_S2, geometry='geometry', crs={
    'proj': 'sinu',
    'R': 6371007.181,
    'units': 'm',
    'no_defs': True
})

df_MODIS['geometry'] = df_MODIS.apply(lambda row: Point(row['x'], row['y']), axis=1)

# Step 2: Convert to GeoDataFrame with Sinusoidal projection
df_MODIS = gpd.GeoDataFrame(df_MODIS, geometry='geometry', crs={
    'proj': 'sinu',
    'R': 6371007.181,
    'units': 'm',
    'no_defs': True
})
df_MODIS
#df_S2

,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,...,srti,ndti,crci,mcrc,savi,ndsvi,x,y,date,geometry
209248,8623.0,10232.0,9768.0,9792.0,9372.0,8907.0,8582.0,9032.0,7894.0,6402.0,...,1.150754,0.070093,-0.955339,-0.953194,-0.060559,-0.953107,-9241585.0,4943775.0,2021-01-13,POINT (-9241585 4943775)
209249,8620.0,9958.0,9620.0,9667.0,9372.0,8907.0,8582.0,8804.0,7894.0,6401.0,...,1.150754,0.070093,-0.955339,-0.953194,-0.070126,-0.953107,-9241575.0,4943775.0,2021-01-13,POINT (-9241575 4943775)
209250,8618.0,9848.0,9560.0,9616.0,9372.0,8907.0,8582.0,8712.0,7894.0,6399.0,...,1.150754,0.070093,-0.955339,-0.953194,-0.073983,-0.953107,-9241565.0,4943775.0,2021-01-13,POINT (-9241565 4943775)
214754,8701.0,10232.0,9677.0,9621.0,9337.0,8869.0,8531.0,8791.0,7861.0,6440.0,...,1.160742,0.074186,-0.954562,-0.952388,-0.067655,-0.952292,-9241605.0,4943765.0,2021-01-13,POINT (-9241605 4943765)
214755,8836.0,10172.0,9634.0,9583.0,9339.0,8871.0,8533.0,8752.0,7863.0,6539.0,...,1.160339,0.074021,-0.954594,-0.952420,-0.068073,-0.952325,-9241595.0,4943765.0,2021-01-13,POINT (-9241595 4943765)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18347921,6142.0,5323.0,6238.0,7520.0,9326.0,9049.0,8603.0,8068.0,7964.0,7488.0,...,1.472461,0.190469,-0.652184,-0.684770,0.055715,-0.718390,-9288375.0,4910835.0,2021-01-13,POINT (-9288375 4910835)
18347922,6084.0,5476.0,6453.0,7617.0,8444.0,8093.0,7851.0,7957.0,7333.0,7435.0,...,1.497419,0.198937,-0.581469,-0.625735,0.033354,-0.664756,-9288365.0,4910835.0,2021-01-13,POINT (-9288365 4910835)
18347923,6084.0,5351.0,6261.0,7149.0,7914.0,7783.0,7479.0,7834.0,7223.0,7435.0,...,1.534004,0.210408,-0.548449,-0.600345,0.070190,-0.642472,-9288355.0,4910835.0,2021-01-13,POINT (-9288355 4910835)
18347924,6084.0,5994.0,6788.0,7582.0,7430.0,7502.0,7138.0,8396.0,7124.0,7435.0,...,1.567141,0.220799,-0.518535,-0.577323,0.077364,-0.622287,-9288345.0,4910835.0,2021-01-13,POINT (-9288345 4910835)


In [9]:
# print(df_S2.crs)
# print(df_MODIS.crs)
df_MODIS

,MODIS_Albedo_WSA_shortwave,x,y,date,geometry
548,479.0,-9243859.354,4.942354e+06,2021-01-13,POINT (-9243859.354 4942354.157)
658,472.0,-9244359.354,4.941854e+06,2021-01-13,POINT (-9244359.354 4941854.157)
767,465.0,-9245359.354,4.941354e+06,2021-01-13,POINT (-9245359.354 4941354.157)
876,510.0,-9246359.354,4.940854e+06,2021-01-13,POINT (-9246359.354 4940854.157)
877,564.0,-9245859.354,4.940854e+06,2021-01-13,POINT (-9245859.354 4940854.157)
...,...,...,...,...,...
7446,504.0,-9291359.354,4.910854e+06,2021-01-13,POINT (-9291359.354 4910854.157)
7447,494.0,-9290859.354,4.910854e+06,2021-01-13,POINT (-9290859.354 4910854.157)
7448,483.0,-9290359.354,4.910854e+06,2021-01-13,POINT (-9290359.354 4910854.157)
7450,634.0,-9289359.354,4.910854e+06,2021-01-13,POINT (-9289359.354 4910854.157)


In [ ]:
joined = gpd.sjoin(df_MODIS,df_S2, how='left', predicate='intersects')
joined
joined["date_right"].unique()

<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[us]

: 

In [26]:
joined.to_csv(r'C:\test\joined_MODIS_S2.csv', index=False)